<a href="https://colab.research.google.com/github/dapopov-st/MiniLlamaSQL/blob/main/Fine_tune_CodeLlama_7b_Instruct_step_by_step.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tune premsql

In [3]:
# import os
# print(os.environ.get('CUDA_HOME'))
# print(os.environ.get('PATH'))
# print(os.environ.get('LD_LIBRARY_PATH'))

In [1]:
import os
import torch
from datasets import load_dataset
from transformers import (
    LlamaForCausalLM,
    LlamaTokenizer,
    AutoModelForCausalLM,
    AutoTokenizer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer


/home/mainuser/anaconda3/envs/sqlft/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-10-08 09:00:40,340] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/mainuser/anaconda3/envs/sqlft/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.3
 [WARNING]  using untested triton version (2.3.1), only 1.0.0 is known to be compatible


In [2]:
print(torch.cuda.is_available())

True


In [3]:
import wandb # do from scipt later
wandb.login()
%env WANDB_PROJECT=sql-fine-tuning

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: dpopovvelasco. Use `wandb login --relogin` to force relogin


env: WANDB_PROJECT=sql-fine-tuning


In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
output_dir = './sql_output_dir_premsql'
logging_dir = './sql_logging_dir_premsql'

In [6]:
import sys,gc,traceback
import torch
def clean_ipython_hist():
    # Code in this function mainly copied from IPython source
    if not 'get_ipython' in globals(): return
    ip = get_ipython()
    user_ns = ip.user_ns
    ip.displayhook.flush()
    pc = ip.displayhook.prompt_count + 1
    for n in range(1, pc): user_ns.pop('_i'+repr(n),None)
    user_ns.update(dict(_i='',_ii='',_iii=''))
    hm = ip.history_manager
    hm.input_hist_parsed[:] = [''] * pc
    hm.input_hist_raw[:] = [''] * pc
    hm._i = hm._ii = hm._iii = hm._i00 =  ''



def clean_tb():
    # h/t Piotr Czapla
    if hasattr(sys, 'last_traceback'):
        traceback.clear_frames(sys.last_traceback)
        delattr(sys, 'last_traceback')
    if hasattr(sys, 'last_type'): delattr(sys, 'last_type')
    if hasattr(sys, 'last_value'): delattr(sys, 'last_value')

def clean_mem():
    clean_tb()
    clean_ipython_hist()
    gc.collect()
    torch.cuda.empty_cache()

In [7]:
clean_mem()

## Load and Prepare the Data

<|begin▁of▁sentence|> Question: What is the total sales amount? Context: The sales table contains information about each sale including date, amount, and customer ID. Answer: SELECT SUM(amount) FROM sales; <|end▁of▁sentence|>

In [10]:

from transformers import AutoTokenizer
from datasets import load_dataset

# Load a tokenizer to use its chat template
template_tokenizer = AutoTokenizer.from_pretrained(
    "premai-io/prem-1B-SQL"
)

# def format_prompt(sample):

#     """Given a sample dictionary with keys "title" and "abstract" format into a prompt.

#     Args:
#       sample: A sample dictionary from a Hugging Face dataset.

#     Returns:
#       sample: sample dictionary with "text" key for the formatted prompt.
#     """
#     #sample['text']=f"[INST] <> Write SQL code to answer the question based on the context. Please wrap your code answer using ```: <> {sample['question']} {sample['context']} [/INST] {sample['answer']}"
#     sample['text']=f"""<|begin▁of▁sentence|> You are a helpful assistant who is a SQL expert. Your task is to write a SQL query based on the given context and question.
# Context: {sample['context']}
# Question: {sample['question']}
# SQL Query:{sample['answer']} <|end▁of▁sentence|>"""
#     return {"text":sample['text']}

def format_prompt(sample):
#     sample['text']=f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
# You are a helpful assistant who is a SQL expert. Your task is to write a SQL query based on the given context and question.<|eot_id|><|start_header_id|>user<|end_header_id|>
# Context: {sample['context']}
# Question: {sample['question']}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
# SQL Query:{sample['answer']} <|end▁of▁sentence|>"""
    sample['text'] = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful assistant who is a SQL expert. Your task is to write a SQL query based on the given context and question.<|eot_id|><|start_header_id|>user<|end_header_id|>

Context: {sample['context']}
Question: {sample['question']}<|eot_id|><|start_header_id|>assistant<|end_header_id|>SQL Query:{sample['answer']} <|end▁of▁sentence|>"""
    return {"text":sample['text']}
  

# Load and format the data using the template TinyLLama is using
dataset = (
    load_dataset("dpv/exoplanets-sql")#, split="train")
      .shuffle(seed=42)
)
dataset = dataset.map(format_prompt)

Map: 100%|██████████| 10/10 [00:00<00:00, 2521.98 examples/s]


In [11]:
dataset

DatasetDict({
    train: Dataset({
        features: ['context', 'question', 'answer', 'results', 'text'],
        num_rows: 50
    })
    valid: Dataset({
        features: ['context', 'question', 'answer', 'results', 'text'],
        num_rows: 10
    })
})

In [11]:
dataset['train']['text'][0]

"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\nYou are a helpful assistant who is a SQL expert. Your task is to write a SQL query based on the given context and question.<|eot_id|><|start_header_id|>user<|end_header_id|>\nContext: CREATE TABLE exoplanets (\n    name TEXT,\n    distance REAL,\n    stellar_magnitude REAL,\n    planet_type TEXT,\n    discovery_year INTEGER,\n    mass_multiplier REAL,\n    mass_wrt TEXT,\n    radius_multiplier REAL,\n    radius_wrt TEXT,\n    orbital_radius REAL,\n    orbital_period REAL,\n    eccentricity REAL,\n    detection_method TEXT\n); CREATE TABLE reference_planets (name TEXT, mass REAL);  'mass_wrt' in exoplanets table has a one-to-one match to 'name' in reference_planets table, and  'name' refers to either Earth or Jupyter, with Jupyter having 317.8 the mass of Earth.\nQuestion: What is the average mass of exoplanets discovered each year?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\nSQL Query:SELECT \n    e.discovery_yea

In [12]:
dataset

DatasetDict({
    train: Dataset({
        features: ['context', 'question', 'answer', 'results', 'text'],
        num_rows: 50
    })
    valid: Dataset({
        features: ['context', 'question', 'answer', 'results', 'text'],
        num_rows: 10
    })
})

- SFTTrainer expect a single 'text' column

## Evaluate the original model

In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer
MODEL_ID="premai-io/prem-1B-SQL"
model = AutoModelForCausalLM.from_pretrained(MODEL_ID).to("cuda:1")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

Unrecognized keys in `rope_scaling` for 'rope_type'='linear': {'type'}
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  6.92it/s]


In [25]:
import sqlite3
import json
import pandas as pd
pd.set_option('display.max_colwidth',1000)
conn = sqlite3.connect('./databases/exoplanets_db.db')
cursor = conn.cursor()
cursor.execute(f"SELECT * FROM exoplanets LIMIT 5;")
def run_query(query, cursor):
    cursor.execute(query)
    rows = cursor.fetchall()
    columns = [desc[0] for desc in cursor.description]
    result_dict = [dict(zip(columns, row)) for row in rows]
    return json.dumps(result_dict)
valid_loaded = load_dataset('json', data_files='./databases/valid.json', split='train')

def generate_sql(context, question, max_new_tokens = 256, debug=False):
    full_question = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful assistant who is a SQL expert. Your task is to write a SQL query based on the given context and question.<|eot_id|><|start_header_id|>user<|end_header_id|>

Context: {context}
Question: {question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>SQL Query:"""
    model_input = tokenizer(full_question, return_tensors="pt").to("cuda:1")
    model.eval()
    with torch.no_grad():
        out = tokenizer.decode(model.generate(**model_input,max_new_tokens=max_new_tokens)[0],skip_special_tokens=True)
    res = out[out.find('SQL Query:')+10:]
    #print(out[out.find(';')])
    if debug: print(res)
    run_query(res, cursor)
    return res
num_opp_errors = 0
num_other_exceptions = 0
op_errors = []
exceptions  = []
for idx,example in enumerate(valid_loaded):
    try:
        print('idx: ',idx)
        generate_sql(context=example['context'], question=example['question'],debug=True)
    except sqlite3.OperationalError as op_err:
        num_opp_errors += 1
        op_errors.append((idx,op_err))
    except Exception as e:
        num_other_exceptions +=1
        exceptions.append((idx,e))
        print(e)
print(f"num_opp_errors: {num_opp_errors}, num_other_exceptions:{num_other_exceptions},\nError rate: {(num_other_exceptions+num_opp_errors)*100/len(valid_loaded)}")

Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


idx:  0


Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


 SELECT discovery_year, COUNT(*) as num_exoplanets FROM exoplanets GROUP BY discovery_year ORDER BY num_exoplanets DESC LIMIT 1;

idx:  1


Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


...

 SELECT name, mass, discovery_year FROM (SELECT name, mass, discovery_year, ROW_NUMBER() OVER (PARTITION BY discovery_year ORDER BY mass ASC) as rn FROM exoplanets) t WHERE rn = 1;  'rn' is the row number assigned by the ROW_NUMBER() OVER (PARTITION BY discovery_year ORDER BY mass ASC) as rn. If rn = 1, then the mass of the exoplanet is the minimum for that discovery year.

idx:  7


Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


 SELECT name, discovery_year, detection_method FROM exoplanets WHERE discovery_year BETWEEN (SELECT MAX(discovery_year) - 2 FROM exoplanets) AND (SELECT MAX(discovery_year) FROM exoplanets);
idx:  8


Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


 SELECT planet_type, AVG(orbital_radius) AS avg_orbital_radius FROM exoplanets GROUP BY planet_type ORDER BY avg_orbital_radius DESC;
idx:  9
 SELECT name, (mass_wrt::REAL / radius_wrt::REAL) AS ratio FROM exoplanets ORDER BY ratio DESC LIMIT 5;  'mass_wrt' and 'radius_wrt' are the mass and radius of the exoplanet in the reference_planets table, respectively.

num_opp_errors: 3, num_other_exceptions:2,
Error rate: 50.0


In [19]:
op_errors, exceptions

([(2, sqlite3.OperationalError('no such column: mass')),
  (6, sqlite3.OperationalError('no such column: mass')),
  (9, sqlite3.OperationalError('unrecognized token: ":"'))],
 [(3,
   sqlite3.ProgrammingError('You can only execute one statement at a time.')),
  (5,
   sqlite3.ProgrammingError('You can only execute one statement at a time.'))])

In [ ]:
# Looks like 

## Configure Model, Data, Tokenizer, Trainer

### LoRA of QLoRA (configured in LoraConfig)
-  lora_alpha=16, # rank of the adaptor matrices, tunable hyperparam; may consider 4 or 8 to reduce number of parameters and speed up computation
-  task_type="CAUSAL_LM" for generative modeling
- Choose which layers to quantize; QLoRA paper had full quantization for intermediate layers while LoRA quantized QV.  Model printout below shows where they are coming from.


#### Q of QLoRA (quantization configured in BitsAndBytesConfig
- Enable Quantization with BitsAndBytesConfig
- Model's weights are frozen and stored in nf4 format, when need to perform compuation, confert to bf16 for stability

Fine-tune the model using a small number of parameters of an adopter layer, making the entire process faster and more memory-efficient.

- Use the following peft cofig to get more adaptor parameters
```python
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM", #for generative modeling
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj","up_proj", "down_proj",'lm_head']
)
```


In [26]:
clean_mem()

In [24]:
# import os
# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [27]:
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_config,get_peft_model
MODEL_ID="premai-io/prem-1B-SQL" # just released Sept. 25, 2024, waiting for access!

bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map='auto',
    #quantization_config = bnb_config
)

model.config.use_cache = False
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
tokenizer.pad_token = "<pad>"
tokenizer.padding_side = "left"
peft_config = LoraConfig(
    lora_alpha=32,
    lora_dropout=0.1,
    r=128,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=  # Layers to target
     ["k_proj", "gate_proj", "v_proj", "up_proj", "q_proj", "o_proj", "down_proj"]
)

# model = prepare_model_for_kbit_training(model)
# model = get_peft_model(model, peft_config)


Unrecognized keys in `rope_scaling` for 'rope_type'='linear': {'type'}
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.74it/s]


In [28]:
from transformers import DataCollatorForLanguageModeling
# Step 3: Launch training with SFTTrainer
training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    num_train_epochs = 10.0,
    logging_steps=10,
    fp16=True,
    gradient_checkpointing=True
)
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset['train'],
    eval_dataset=dataset['valid'],
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
   # peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=256,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=True,
)

trainer.train()


Generating train split: 86 examples [00:00, 542.35 examples/s]
Generating train split: 16 examples [00:00, 6050.20 examples/s]
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


 10%|█         | 10/100 [00:09<01:18,  1.14it/s]

{'loss': 0.9196, 'grad_norm': 3.8430662155151367, 'learning_rate': 0.00019510565162951537, 'epoch': 0.93}


 20%|██        | 20/100 [00:18<01:09,  1.15it/s]

{'loss': 0.2243, 'grad_norm': 1.5269800424575806, 'learning_rate': 0.00018090169943749476, 'epoch': 1.86}


...

100%|██████████| 100/100 [01:27<00:00,  1.15it/s]

{'loss': 0.0124, 'grad_norm': 0.2706953287124634, 'learning_rate': 1.973271571728441e-07, 'epoch': 9.3}


100%|██████████| 100/100 [01:40<00:00,  1.00s/it]

{'train_runtime': 100.7915, 'train_samples_per_second': 8.532, 'train_steps_per_second': 0.992, 'train_loss': 0.170979051142931, 'epoch': 9.3}


TrainOutput(global_step=100, training_loss=0.170979051142931, metrics={'train_runtime': 100.7915, 'train_samples_per_second': 8.532, 'train_steps_per_second': 0.992, 'total_flos': 1573369833062400.0, 'train_loss': 0.170979051142931, 'epoch': 9.30232558139535})

In [30]:
import sqlite3
import json
import pandas as pd
pd.set_option('display.max_colwidth',1000)
conn = sqlite3.connect('./databases/exoplanets_db.db')
cursor = conn.cursor()
cursor.execute(f"SELECT * FROM exoplanets LIMIT 5;")
def run_query(query, cursor):
    cursor.execute(query)
    rows = cursor.fetchall()
    columns = [desc[0] for desc in cursor.description]
    result_dict = [dict(zip(columns, row)) for row in rows]
    return json.dumps(result_dict)
valid_loaded = load_dataset('json', data_files='./databases/valid.json', split='train')

def generate_sql(context, question, max_new_tokens = 256, debug=False):
    full_question = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful assistant who is a SQL expert. Your task is to write a SQL query based on the given context and question.<|eot_id|><|start_header_id|>user<|end_header_id|>

Context: {context}
Question: {question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>SQL Query:"""
    model_input = tokenizer(full_question, return_tensors="pt").to("cuda:1")
    model.eval()
    with torch.no_grad():
        out = tokenizer.decode(model.generate(**model_input,max_new_tokens=max_new_tokens)[0],skip_special_tokens=True)
    res = out[out.find('SQL Query:')+10:out.find('<|end▁of▁sentence|>')]
    #print(out[out.find(';')])
    if debug: print(res)
    run_query(res, cursor)
    return res
num_opp_errors = 0
num_other_exceptions = 0
op_errors = []
exceptions  = []
for idx,example in enumerate(valid_loaded):
    try:
        print('idx: ',idx)
        generate_sql(context=example['context'], question=example['question'],debug=True)
    except sqlite3.OperationalError as op_err:
        num_opp_errors += 1
        op_errors.append((idx,op_err))
    except Exception as e:
        num_other_exceptions +=1
        exceptions.append((idx,e))
        print(e)
print(f"num_opp_errors: {num_opp_errors}, num_other_exceptions:{num_other_exceptions},\nError rate: {(num_other_exceptions+num_opp_errors)*100/len(valid_loaded)}")

Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


idx:  0


Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


WITH YearlyCounts AS (
    SELECT 
        discovery_year,
        COUNT(*) AS num_discoveries
    FROM exoplanets
    GROUP BY discovery_year
)

SELECT 
    discovery_year,
    num_discoveries,
    AVG(num_discoveries) OVER (ORDER BY discovery_year ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS moving_avg
FROM YearlyCounts
ORDER BY discovery_year; 
idx:  1


Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


...

WITH MinMassExoplanets AS (
    SELECT 
        e.name,
        e.mass_multiplier * r.mass AS mass,
        e.discovery_year,
        r.name AS reference_planet,
        r.mass_multiplier * r.mass AS mass
    FROM exoplanets e
JOIN reference_planets r ON e.mass_wrt = r.name
WHERE e.mass_multiplier * r.mass < (SELECT MIN(mass) FROM MinMassExoplanets)
ORDER BY mass DESC; 
idx:  7


Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


SELECT e.name, e.distance, e.stellar_magnitude
FROM exoplanets e
WHERE e.discovery_year >= strftime('%Y', 'now') - 2  -- Last 2 years
ORDER BY e.discovery_year DESC; 
idx:  8


Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


SELECT planet_type, AVG(orbital_radius) AS avg_orbital_radius
FROM exoplanets
GROUP BY planet_type
ORDER BY avg_orbital_radius DESC; 
idx:  9
WITH RankedExoplanets AS (
    SELECT 
        e.name,
        e.mass_multiplier * r.mass AS mass,
        e.discovery_year,
        RANK() OVER (PARTITION BY e.discovery_year ORDER BY e.mass_multiplier * r.mass DESC) AS mass_rank
    FROM exoplanets e
JOIN reference_planets r ON e.mass_wrt = r.name
WHERE mass_rank <= 5; 
num_opp_errors: 2, num_other_exceptions:1,
Error rate: 30.0


## Merging the model

In [72]:
# from peft import PeftModel

# # Load the fine-tuned model with LoRA
# #model = AutoModelForCausalLM.from_pretrained(script_args.model_name)

# # Load the LoRA adapter (replace 'path_to_your_adapter' with your path)
# lora_model = PeftModel.from_pretrained(model, script_args.output_dir)

# # Merge the LoRA weights with the base model
# merged_model = lora_model.merge_and_unload()
# merged_model.save_pretrained(f'{output_dir}/merged_model_1000')


In [ ]:
clean_mem()

In [73]:
# output_dir = os.path.join(output_dir, "final_checkpoints")
# trainer.model.save_pretrained(output_dir)

# # Free memory for merging weights
# del model
# torch.cuda.empty_cache()

# from peft import AutoPeftModelForCausalLM

# model = AutoPeftModelForCausalLM.from_pretrained(output_dir, device_map="auto", torch_dtype=torch.bfloat16)
# model = model.merge_and_unload()

# output_merged_dir = os.path.join(script_args.output_dir, "final_merged_checkpoint")
# model.save_pretrained(output_merged_dir, safe_serialization=True)

In [74]:
merged_model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072)
    (layers): ModuleList(
      (0-27): 28 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
      )
    )
    (norm

- The way of loading below seemed to produce good results

In [ ]:
from peft import AutoPeftModelForCausalLM
output_merged_dir = os.path.join(output_dir, "final_merged_checkpoint")

model = AutoPeftModelForCausalLM.from_pretrained(output_dir, device_map="auto", torch_dtype=torch.bfloat16)

In [62]:
# MODEL_ID='codellama/CodeLlama-7b-Instruct-hf'
# base_model_id = MODEL_ID
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16
# )


# tokenizer = AutoTokenizer.from_pretrained(
#     base_model_id,
#     add_bos_token=True,
#     trust_remote_code=True,
# )



# def format_eval_prompt(sample):
#     #sample['text']=f"[INST] <> Write SQL code to answer the question based on the context. Please wrap your code answer using ```: <> {sample['question']} {sample['context']} [/INST] {sample['answer']}"
#     return f"""<s>You are a helpful assistant who is a SQL expert. Your task is to write a SQL query based on the given context and question.  Do not provide a query explanation, just a correct answer.
# Context: {sample['context']}
# Question: {sample['question']}
# SQL Query:
# """

# question = 'Which Class has a Frequency MHz larger than 91.5, and a City of license of hyannis, nebraska?'
# context = 'CREATE TABLE table_name_12 (class VARCHAR, frequency_mhz VARCHAR, city_of_license VARCHAR)'



def format_eval_prompt(sample):
    
    return f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful assistant who is a SQL expert. Your task is to write a SQL query based on the given context and question.<|eot_id|><|start_header_id|>user<|end_header_id|>

Context: {sample['context']}
Question: {sample['question']}<|eot_id|><|start_header_id|>assistant<|end_header_id|>SQL Query:"""


eval_prompt = format_eval_prompt(dataset['test'][1])


In [63]:
eval_prompt

"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a helpful assistant who is a SQL expert. Your task is to write a SQL query based on the given context and question.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nContext: CREATE TABLE Programs (ProgramID int, ProgramName varchar(255)); INSERT INTO Programs (ProgramID, ProgramName) VALUES (1, 'Education'), (2, 'Health'), (3, 'Environment'); CREATE TABLE Donations (DonationID int, DonationAmount decimal(10,2), DonationDate date, ProgramID int); INSERT INTO Donations (DonationID, DonationAmount, DonationDate, ProgramID) VALUES (1, 500, '2021-07-01', 2), (2, 300, '2021-09-05', 2), (3, 700, '2021-10-30', 3);\nQuestion: What was the total donation amount for the 'Health' program in 2021?<|eot_id|><|start_header_id|>assistant<|end_header_id|>SQL Query:"

In [67]:
model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=150)[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


system

You are a helpful assistant who is a SQL expert. Your task is to write a SQL query based on the given context and question.user

Context: CREATE TABLE Programs (ProgramID int, ProgramName varchar(255)); INSERT INTO Programs (ProgramID, ProgramName) VALUES (1, 'Education'), (2, 'Health'), (3, 'Environment'); CREATE TABLE Donations (DonationID int, DonationAmount decimal(10,2), DonationDate date, ProgramID int); INSERT INTO Donations (DonationID, DonationAmount, DonationDate, ProgramID) VALUES (1, 500, '2021-07-01', 2), (2, 300, '2021-09-05', 2), (3, 700, '2021-10-30', 3);
Question: What was the total donation amount for the 'Health' program in 2021?assistantSQL Query:SELECT SUM(T2.DonationAmount) FROM Programs AS T1 INNER JOIN Donations AS T2 ON T1.ProgramID = T2.ProgramID WHERE T1.ProgramName = 'Health' AND YEAR(T2.DonationDate) = 2021;


In [78]:
merged_model.to('cuda:1')

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072)
    (layers): ModuleList(
      (0-27): 28 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
      )
    )
    (norm

In [80]:
model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda:1")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(merged_model.generate(**model_input, max_new_tokens=150)[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


system

You are a helpful assistant who is a SQL expert. Your task is to write a SQL query based on the given context and question.user

Context: CREATE TABLE Programs (ProgramID int, ProgramName varchar(255)); INSERT INTO Programs (ProgramID, ProgramName) VALUES (1, 'Education'), (2, 'Health'), (3, 'Environment'); CREATE TABLE Donations (DonationID int, DonationAmount decimal(10,2), DonationDate date, ProgramID int); INSERT INTO Donations (DonationID, DonationAmount, DonationDate, ProgramID) VALUES (1, 500, '2021-07-01', 2), (2, 300, '2021-09-05', 2), (3, 700, '2021-10-30', 3);
Question: What was the total donation amount for the 'Health' program in 2021?assistantSQL Query: 

SELECT SUM(DonationAmount) FROM Donations WHERE ProgramID = 2 AND DonationDate BETWEEN '2021-01-01' AND '2021-12-31'; 

Explanation: This SQL query is a SELECT statement with a WHERE clause. The SELECT statement is used to retrieve the SUM of the DonationAmount from the Donations table. The WHERE clause is used t

In [66]:
dataset['test'][1]['answer']

"SELECT SUM(DonationAmount) FROM Donations WHERE ProgramID = (SELECT ProgramID FROM Programs WHERE ProgramName = 'Health') AND DonationDate BETWEEN '2021-01-01' AND '2021-12-31';"

In [32]:
# question = "What are the distinct creation years of the departments managed by a secretary born in state 'Alabama'?"
# context = 'CREATE TABLE department (creation VARCHAR, department_id VARCHAR); CREATE TABLE management (department_id VARCHAR, head_id VARCHAR); CREATE TABLE head (head_id VARCHAR, born_state VARCHAR)'
# ref_answer ="SELECT DISTINCT T1.creation FROM department AS T1 JOIN management AS T2 ON T1.department_id = T2.department_id JOIN head AS T3 ON T2.head_id = T3.head_id WHERE T3.born_state = 'Alabama'"
# eval_prompt = format_eval_prompt(question,context)
# model_input = eval_tokenizer(eval_prompt, return_tensors="pt").to("cuda")

# model.eval()
# with torch.no_grad():
#     print(eval_tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))


- Now load with the original bnb config after restarting nb to clean memory

In [ ]:
from peft import AutoPeftModelForCausalLM
output_merged_dir = os.path.join(output_dir, "final_merged_checkpoint")
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True,
)
model = AutoPeftModelForCausalLM.from_pretrained(output_dir, device_map="auto", quantization_config=bnb_config,torch_dtype=torch.bfloat16)

- Model loaded with bnb is about 2.5x smaller

In [ ]:
model

In [47]:
MODEL_ID='codellama/CodeLlama-7b-Instruct-hf'
base_model_id = MODEL_ID
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# base_model = AutoModelForCausalLM.from_pretrained(
#     base_model_id,  # Mistral, same as before
#     quantization_config=bnb_config,  # Same quantization config as before
#     device_map="auto",
#     trust_remote_code=True,
#     use_auth_token=True
# )

eval_tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    add_bos_token=True,
    trust_remote_code=True,
)


eval_prompt = """You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.

You must output the SQL query that answers the question.
### Input:
Which Class has a Frequency MHz larger than 91.5, and a City of license of hyannis, nebraska?

### Context:
CREATE TABLE table_name_12 (class VARCHAR, frequency_mhz VARCHAR, city_of_license VARCHAR)

### Response:
"""
def format_eval_prompt(question,context):
  return f"<s>[INST] <<SYS>> Write SQL code to answer the question based on the context. Please wrap your code answer using ```: <</SYS>> {question} {context} [/INST]"

question = 'Which Class has a Frequency MHz larger than 91.5, and a City of license of hyannis, nebraska?'
context = 'CREATE TABLE table_name_12 (class VARCHAR, frequency_mhz VARCHAR, city_of_license VARCHAR)'

eval_prompt = format_eval_prompt(question,context)
model_input = eval_tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(eval_tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[INST] <<SYS>> Write SQL code to answer the question based on the context. Please wrap your code answer using ```: <</SYS>> Which Class has a Frequency MHz larger than 91.5, and a City of license of hyannis, nebraska? CREATE TABLE table_name_12 (class VARCHAR, frequency_mhz VARCHAR, city_of_license VARCHAR) [/INST] <<SYS doc Write SQL code to answer the question based on the context. Please wrap your code answer using ```:ле code answer using ```:ле</SYS doc Which Class has a Frequency MHz larger than 91.5, and a City of license of hyannis, nebraska CREATE TABLE table_name_1	ón (class VARCHAR, frequency_mhz VARCHAR, city_of_license VARCHAR) [/INST] <<SYS


In [68]:
# question = "What are the distinct creation years of the departments managed by a secretary born in state 'Alabama'?"
# context = 'CREATE TABLE department (creation VARCHAR, department_id VARCHAR); CREATE TABLE management (department_id VARCHAR, head_id VARCHAR); CREATE TABLE head (head_id VARCHAR, born_state VARCHAR)'
# ref_answer ="SELECT DISTINCT T1.creation FROM department AS T1 JOIN management AS T2 ON T1.department_id = T2.department_id JOIN head AS T3 ON T2.head_id = T3.head_id WHERE T3.born_state = 'Alabama'"
# eval_prompt = format_eval_prompt(question,context)
# model_input = eval_tokenizer(eval_prompt, return_tensors="pt").to("cuda")

# model.eval()
# with torch.no_grad():
#     print(eval_tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))


In [ ]:
#Likely need to restart kernel after training since o.w. run out of CUDA memory
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
MODEL_ID='codellama/CodeLlama-7b-Instruct-hf'
base_model_id = MODEL_ID
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,  # Mistral, same as before
    quantization_config=bnb_config,  # Same quantization config as before
    device_map="auto",
    trust_remote_code=True,
    use_auth_token=True
)

eval_tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    add_bos_token=True,
    trust_remote_code=True,
)

#ft_model = PeftModel.from_pretrained(base_model, output_dir+'/'+'checkpoint-10')
#ft_model = PeftModel.from_pretrained(base_model, output_dir+'/'+'checkpoint-4')
ft_model = PeftModel.from_pretrained(base_model, output_dir+'/'+'checkpoint-40')


In [69]:
# eval_prompt = """You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.

# You must output the SQL query that answers the question.
# ### Input:
# Which Class has a Frequency MHz larger than 91.5, and a City of license of hyannis, nebraska?

# ### Context:
# CREATE TABLE table_name_12 (class VARCHAR, frequency_mhz VARCHAR, city_of_license VARCHAR)

# ### Response:
# """
# def format_eval_prompt(question,context):
#   return f"<s>[INST] <<SYS>> Write SQL code to answer the question based on the context. Please wrap your code answer using ```: <</SYS>> {question} {context} [/INST]"

# question = 'Which Class has a Frequency MHz larger than 91.5, and a City of license of hyannis, nebraska?'
# context = 'CREATE TABLE table_name_12 (class VARCHAR, frequency_mhz VARCHAR, city_of_license VARCHAR)'

# eval_prompt = format_eval_prompt(question,context)
# model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

# model.eval()
# with torch.no_grad():
#     print(eval_tokenizer.decode(base_model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))

# model.eval()
# with torch.no_grad():
#     print(eval_tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))

In [ ]:
clean_mem()

In [ ]:
question = "What are the distinct creation years of the departments managed by a secretary born in state 'Alabama'?"
context = 'CREATE TABLE department (creation VARCHAR, department_id VARCHAR); CREATE TABLE management (department_id VARCHAR, head_id VARCHAR); CREATE TABLE head (head_id VARCHAR, born_state VARCHAR)'
ref_answer ="SELECT DISTINCT T1.creation FROM department AS T1 JOIN management AS T2 ON T1.department_id = T2.department_id JOIN head AS T3 ON T2.head_id = T3.head_id WHERE T3.born_state = 'Alabama'"
eval_prompt = format_eval_prompt(question,context)
model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

base_model.eval()
with torch.no_grad():
    print(eval_tokenizer.decode(base_model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))

ft_model.eval()
with torch.no_grad():
    print(eval_tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))

model.eval()
with torch.no_grad():
    print(eval_tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))


- OK, not that great yet...

In [ ]:
# adaptor_name = "CodeLlama7bInstruct-SQLAdaptor"
# HUGGINGFACE_USER_NAME = "dpv"

# model.push_to_hub(f"{HUGGINGFACE_USER_NAME}/{adaptor_name}", use_auth_token=True)

## Reloading After Train

### Reloading with bnb_config since that's how the model was trained

In [ ]:
from peft import AutoPeftModelForCausalLM
# compute_dtype = getattr(torch, "float16")
args_ = {
    "use_4bit": True,
    "use_nested_quant": False,
    "bnb_4bit_compute_dtype": "float16",
    "bnb_4bit_quant_type": "nf4",
}
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=False,
)

# Path to the directory where the model was saved
model_dir = output_dir+"/final_checkpoints"

# Load the model
model = AutoPeftModelForCausalLM.from_pretrained(model_dir, quantization_config=bnb_config)

In [ ]:

def format_eval_prompt(question,context):
  return f"<s>[INST] <<SYS>> Write SQL code to answer the question based on the context. Please wrap your code answer using ```: <</SYS>> {question} {context} [/INST]"

question = 'Which Class has a Frequency MHz larger than 91.5, and a City of license of hyannis, nebraska?'
context = 'CREATE TABLE table_name_12 (class VARCHAR, frequency_mhz VARCHAR, city_of_license VARCHAR)'

eval_prompt = format_eval_prompt(question,context)

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
MODEL_ID='codellama/CodeLlama-7b-Instruct-hf'
base_model_id = MODEL_ID


tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    add_bos_token=True,
    trust_remote_code=True,
)
model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))


In [24]:
question = "What are the distinct creation years of the departments managed by a secretary born in state 'Alabama'?"
context = 'CREATE TABLE department (creation VARCHAR, department_id VARCHAR); CREATE TABLE management (department_id VARCHAR, head_id VARCHAR); CREATE TABLE head (head_id VARCHAR, born_state VARCHAR)'
ref_answer ="SELECT DISTINCT T1.creation FROM department AS T1 JOIN management AS T2 ON T1.department_id = T2.department_id JOIN head AS T3 ON T2.head_id = T3.head_id WHERE T3.born_state = 'Alabama'"
eval_prompt = format_eval_prompt(question,context)
model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")


model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=150)[0], skip_special_tokens=True))


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<s>[INST] <<SYS>> Write SQL code to answer the question based on the context. Please wrap your code answer using ```: <</SYS>> What are the distinct creation years of the departments managed by a secretary born in state 'Alabama'? CREATE TABLE department (creation VARCHAR, department_id VARCHAR); CREATE TABLE management (department_id VARCHAR, head_id VARCHAR); CREATE TABLE head (head_id VARCHAR, born_state VARCHAR) [/INST] 

## Step 1: Identify the necessary tables and their relationships
To solve this problem, we need to identify the necessary tables and their relationships. We have three tables: department, management, and head. The department table stores information about each department, the management table stores information about the departments and their heads, and the head table stores information about each head, including their born state.

## Step 2: Determine the conditions for the query
We are asked to find the distinct creation years of the departments managed by a sec

In [ ]:
context="CREATE TABLE diagnoses ( subject_id VARCHAR, hadm_id VARCHAR, icd9_code VARCHAR, short_title VARCHAR, long_title VARCHAR ); CREATE TABLE demographic ( subject_id VARCHAR, hadm_id VARCHAR, name VARCHAR, marital_status VARCHAR, age VARCHAR, dob VARCHAR, gender VARCHAR, language VARCHAR, religion VARCHAR, admission_type VARCHAR, days_stay VARCHAR, insurance VARCHAR, ethnicity VARCHAR, expire_flag VARCHAR, admission_location VARCHAR, discharge_location VARCHAR, diagnosis VARCHAR, dod VARCHAR, dob_year VARCHAR, dod_year VARCHAR, admittime VARCHAR, dischtime VARCHAR, admityear VARCHAR ); CREATE TABLE lab ( subject_id VARCHAR, hadm_id VARCHAR, itemid VARCHAR, charttime VARCHAR, flag VARCHAR, value_unit VARCHAR, label VARCHAR, fluid VARCHAR ); CREATE TABLE prescriptions ( subject_id VARCHAR, hadm_id VARCHAR, icustay_id VARCHAR, drug_type VARCHAR, drug VARCHAR, formulary_drug_cd VARCHAR, route VARCHAR, drug_dose VARCHAR ); CREATE TABLE procedures ( subject_id VARCHAR, hadm_id VARCHAR, icd9_code VARCHAR, short_title VARCHAR, long_title VARCHAR )"
question="Count the number of patients whose insurance is government and procedure short title is rt/left heart card cath?"
ref_answer = 'SELECT COUNT(DISTINCT demographic.subject_id) FROM demographic INNER JOIN procedures ON demographic.hadm_id = procedures.hadm_id WHERE demographic.insurance = "Government" AND procedures.short_title = "Rt/left heart card cath"'
eval_prompt = format_eval_prompt(question,context)
model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")


model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=150)[0], skip_special_tokens=True))


    Part 1
    Load the base model

    Load and activate the adapter on top of the base model

    Merge the adapter with the base model

    Save the merged model

    Part 2
    Remember, the adapter is fine-tuned with QLoRA on top of a quantized model. So maybe we just need to quantize the merged model with the same QLoRA configuration before using it for inference.
    
    Source: https://kaitchup.substack.com/p/lora-adapters-when-a-naive-merge?source=post_page-----3204119f0426--------------------------------


In [ ]:
# adaptor = f"{HUGGINGFACE_USER_NAME}/{adaptor_name}"
# config = PeftConfig.from_pretrained(adaptor)

# #Load the base model with default precision
# model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path)

# #Load and activate the adapter on top of the base model
# model = PeftModel.from_pretrained(model, adaptor)

# #Merge the adaptor with the base model
# model = model.merge_and_unload()

# #Save the merged model in a directory "./naive_merge/" in the safetensors format
# model.save_pretrained("./naive_merge/", safe_serialization=True)

# # Part 2: Quantize the merged model before using it for inference

# compute_dtype = getattr(torch, "float16")
# bnb_config = BitsAndBytesConfig(
#         load_in_4bit=True,
#         bnb_4bit_quant_type="nf4",
#         bnb_4bit_compute_dtype=compute_dtype,
#         bnb_4bit_use_double_quant=True,
# )
# model = AutoModelForCausalLM.from_pretrained("./naive_merge/", quantization_config=bnb_config, device_map={"": 0})
# model.save_pretrained(output_dir, safe_serialization=True)



# # peft_model_id = f"{HUGGINGFACE_USER_NAME}/{model_name}"
# # config = PeftConfig.from_pretrained(peft_model_id)
# # #model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=False)
# # tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# # # Load the Lora model
# # model = PeftModel.from_pretrained(model, peft_model_id)

In [ ]:
adaptor = f"{HUGGINGFACE_USER_NAME}/{adaptor_name}"
config = PeftConfig.from_pretrained(adaptor)
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True,
)

#Load the base model with default precision
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path,
                                             quantization_config=bnb_config, device_map={"": 0})

#Load and activate the adapter on top of the base model
model = PeftModel.from_pretrained(model, adaptor)

#Merge the adaptor with the base model
model = model.merge_and_unload()

#Save the merged model in a directory "./naive_merge/" in the safetensors format
model.save_pretrained(output_dir, safe_serialization=True)


# peft_model_id = f"{HUGGINGFACE_USER_NAME}/{model_name}"
# config = PeftConfig.from_pretrained(peft_model_id)
# #model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=False)
# tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# # Load the Lora model
# model = PeftModel.from_pretrained(model, peft_model_id)

## Merging Take 2

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir results/runs

In [ ]:
clean_mem()

In [ ]:
# May be able to load my model shards as below

# output_merged_dir = os.path.join(output_dir, "final_merged_checkpoint")
# MODEL_ID='codellama/CodeLlama-7b-Instruct-hf'
# base_model_id = MODEL_ID
# model = LlamaForCausalLM.from_pretrained(MODEL_ID)
# model.base_model = LlamaForCausalLM.from_pretrained(MODEL_ID)
# model.base_model.load_pretrained(output_merged_dir, safe_serialization=True)

In [ ]:
#!model.push_to_hub(trainer.model, use_temp_dir=False)
#!tokenizer.push_to_hub(trainer.model, use_temp_dir=False)